# OSSF demo

In this notebook, we will demonstrate how **Legend** and **Morphir** models can be interpreted on databricks with minimum development overhead. We will be sourcing information with **Legend** and feed the resulting dataframe into a **Morphir** calculation.

Although we could directly query the underlying tables through the spark SQL API and its multiple joins and transformations (see below), we can benefit from the model created through the **Legend** framework and access its logical representation (mapping to a legend entity).

In [0]:
%sql
SELECT
  a.`id`,
  a.`currency`,
  a.`reporting_entity`,
  a.`product`,
  a.`sub_product`,
  a.`market_value`,
  a.`lendable_value`,
  a.`forward_start_amount`,
  a.`forward_start_bucket`,
  a.`collateral_class`,
  a.`treasury_control`,
  a.`accounting_designation`,
  a.`encumbrance_type`,
  a.`internal_counterparty`,
  a.`business_line`,
  m.`maturity_date`,
  m.`purchase_date`
FROM inflows.assets a
JOIN inflows.assets_maturity m
LIMIT 10

id,currency,reporting_entity,product,sub_product,market_value,lendable_value,forward_start_amount,forward_start_bucket,collateral_class,treasury_control,accounting_designation,encumbrance_type,internal_counterparty,business_line,maturity_date,purchase_date
0,CAD,entity_1,UnencumberedAssets,Currency and Coin,1444320.0,1069776.0,null,null,a_0_Q,true,designation_1,encumbrance_1,counterparty_1,business_1,2023-01-01,2022-12-01
1,USD,entity_2,Capacity,Level 1,3194640.0,3143232.0,2970648.0,2,a_1_Q,true,designation_2,encumbrance_2,counterparty_2,business_2,2022-12-01,2022-12-01
2,CHF,entity_3,UnrestrictedReserveBalances,Level 2a,2460240.0,2373336.0,null,null,a_2_Q,false,designation_3,encumbrance_3,counterparty_3,business_3,2022-12-27,2022-12-04
3,EUR,entity_4,RestrictedReserveBalances,Level 2b,3733200.0,3684240.0,null,null,a_3_Q,true,designation_4,encumbrance_4,counterparty_4,business_4,2022-12-29,2022-12-01
4,CHF,entity_1,UnsettledAssetPurchases,Non-HQLA,3769920.0,3552048.0,3350088.0,5,a_4_Q,true,designation_1,null,counterparty_1,business_1,2022-12-23,2022-12-01
5,JPY,entity_2,ForwardAssetPurchases,No Collateral Pledged,1358640.0,1341504.0,946152.0,6,a_5_Q,false,designation_2,encumbrance_2,counterparty_2,business_2,2022-12-23,2022-12-04
6,USD,entity_3,EncumberedAssets,Rehypothecateable Collateral Unencumbered,477360.0,363528.0,140760.0,7,s_1_Q,true,designation_3,encumbrance_3,counterparty_3,business_3,2023-01-01,2022-12-04
7,CAD,entity_4,UnencumberedAssets,Unsettled (Regular Way),3206880.0,3157920.0,2892312.0,8,s_2_Q,true,designation_4,encumbrance_4,counterparty_4,business_4,2023-01-02,2022-12-01
8,USD,entity_1,Capacity,Unsettled (Forward),3414960.0,3205656.0,2959632.0,9,s_3_Q,true,designation_1,encumbrance_1,counterparty_1,business_1,2022-12-21,2022-12-04
9,USD,entity_2,UnrestrictedReserveBalances,firm long,1578960.0,1177488.0,826200.0,10,s_4_Q,true,designation_2,encumbrance_2,counterparty_2,business_2,2022-12-18,2022-12-01


## LEGEND
With our **Legend** model packaged as JAR and included as a cluster dependency, we can easily access each of its underlying entities, create tables programmatically or execute queries according to legend specifications. We show below how to load our model and access a given entity (a logical model mapped to a physical table)

In [0]:
%scala
import org.finos.legend.spark.LegendClasspathLoader
val legend = LegendClasspathLoader.loadResources()

import org.finos.legend.spark.LegendClasspathLoader
legend: org.finos.legend.spark.Legend = org.finos.legend.spark.Legend@4ec3ba85

In [0]:
%scala
legend.getEntityNames.foreach(println)

lcr::entities::product
lcr::services::getInflows
lcr::lakehouse::finos
lcr::entities::collateralClass
lcr::services::getInflowsWithBuckets
lcr::lakehouse::assetMapping
lcr::entities::currency
lcr::entities::asset
lcr::lakehouse::store
lcr::lakehouse::databricks
lcr::functions::computeMaturityBucket

In [0]:
%scala
legend.getSchema("lcr::entities::asset").fields.foreach(println)

StructField(id,IntegerType,false)
StructField(currency,StringType,false)
StructField(converted,BooleanType,false)
StructField(reportingEntity,StringType,false)
StructField(product,StringType,false)
StructField(subProduct,StringType,true)
StructField(marketValue,LongType,false)
StructField(lendableValue,LongType,false)
StructField(purchaseDate,DateType,false)
StructField(maturityDate,DateType,false)
StructField(forwardStartAmount,LongType,true)
StructField(forwardStartBucket,IntegerType,true)
StructField(collateralClass,StringType,false)
StructField(treasuryControl,BooleanType,false)
StructField(accountingDesignation,StringType,true)
StructField(effectiveMaturityDate,DateType,false)
StructField(encumbranceType,StringType,true)
StructField(internalCounterparty,StringType,true)
StructField(businessLine,StringType,false)

Accessing the underlying generated SQL code from pure to databricks SQL...

In [0]:
%scala
println(legend.generateSql("lcr::services::getInflows"))

select `root`.product as `product`, `root`.sub_product as `subProduct`, `root`.collateral_class as `collateralClass`, `root`.market_value as `marketValue`, `assets_maturity_0`.maturity_date as `maturityDate`, `root`.encumbrance_type as `encumbranceType`, `root`.forward_start_amount as `forwardStartAmount`, `root`.forward_start_bucket as `forwardStartBucket`, `root`.treasury_control as `treasuryControl` from inflows.assets as `root` left outer join inflows.assets_maturity as `assets_maturity_0` on (`root`.id = `assets_maturity_0`.id)

... Or executing query directly resulting in a dataframe with both its technical and business constraints enforced (such as multiplicity constraints or enumerations)

In [0]:
%scala
val assetMapping = legend.query("lcr::services::getInflows")
display(assetMapping.limit(10))

product,subProduct,collateralClass,marketValue,maturityDate,encumbranceType,forwardStartAmount,forwardStartBucket,treasuryControl
UnencumberedAssets,Currency and Coin,a_0_Q,1444320.0,2022-12-01,encumbrance_1,null,null,true
Capacity,Level 1,a_1_Q,3194640.0,2022-12-06,encumbrance_2,2970648.0,2,true
UnrestrictedReserveBalances,Level 2a,a_2_Q,2460240.0,2023-01-03,encumbrance_3,null,null,false
RestrictedReserveBalances,Level 2b,a_3_Q,3733200.0,2023-01-03,encumbrance_4,null,null,true
UnsettledAssetPurchases,Non-HQLA,a_4_Q,3769920.0,2022-12-28,null,3350088.0,5,true
ForwardAssetPurchases,No Collateral Pledged,a_5_Q,1358640.0,2022-12-26,encumbrance_2,946152.0,6,false
EncumberedAssets,Rehypothecateable Collateral Unencumbered,s_1_Q,477360.0,2023-01-04,encumbrance_3,140760.0,7,true
UnencumberedAssets,Unsettled (Regular Way),s_2_Q,3206880.0,2022-12-01,encumbrance_4,2892312.0,8,true
Capacity,Unsettled (Forward),s_3_Q,3414960.0,2022-12-04,encumbrance_1,2959632.0,9,true
UnrestrictedReserveBalances,firm long,s_4_Q,1578960.0,2022-12-19,encumbrance_2,826200.0,10,true


We may realize that our output dataframe may not fully comply with **Morphir** specifications (maturity date should be a bucket rather than a date). We can easily get back to legend and create a new service where all the necessary transformations are created (and unit tested) to transform raw data into **Morphir** ready data assets. By doing so, we comply with the "data contract" implied by the **Morphir** framework with no development overhead. An example is the `maturityBucket` that is derived from `transactionDate` and `maturityDate`. Similarly to logical mapping, we can access its underlying generated SQL...

In [0]:
%scala
println(legend.generateSql("lcr::services::getInflowsWithBuckets"))

select `root`.product as `product`, `root`.sub_product as `subProduct`, `root`.collateral_class as `collateralClass`, `root`.market_value as `marketValue`, datediff(`assets_maturity_0`.maturity_date, `assets_maturity_0`.purchase_date) as `maturityBucket`, `root`.encumbrance_type as `encumbranceType`, `root`.forward_start_amount as `forwardStartAmount`, `root`.forward_start_bucket as `forwardStartBucket`, `root`.treasury_control as `treasuryControl` from inflows.assets as `root` left outer join inflows.assets_maturity as `assets_maturity_0` on (`root`.id = `assets_maturity_0`.id)

... or directly execute our query resulting in a dataframe

In [0]:
%scala
val inflows = legend.query("lcr::services::getInflowsWithBuckets")
display(inflows.limit(10))

product,subProduct,collateralClass,marketValue,maturityBucket,encumbranceType,forwardStartAmount,forwardStartBucket,treasuryControl
UnencumberedAssets,Currency and Coin,a_0_Q,1444320.0,31,encumbrance_1,null,null,true
Capacity,Level 1,a_1_Q,3194640.0,0,encumbrance_2,2970648.0,2,true
UnrestrictedReserveBalances,Level 2a,a_2_Q,2460240.0,23,encumbrance_3,null,null,false
RestrictedReserveBalances,Level 2b,a_3_Q,3733200.0,28,encumbrance_4,null,null,true
UnsettledAssetPurchases,Non-HQLA,a_4_Q,3769920.0,22,null,3350088.0,5,true
ForwardAssetPurchases,No Collateral Pledged,a_5_Q,1358640.0,19,encumbrance_2,946152.0,6,false
EncumberedAssets,Rehypothecateable Collateral Unencumbered,s_1_Q,477360.0,28,encumbrance_3,140760.0,7,true
UnencumberedAssets,Unsettled (Regular Way),s_2_Q,3206880.0,32,encumbrance_4,2892312.0,8,true
Capacity,Unsettled (Forward),s_3_Q,3414960.0,17,encumbrance_1,2959632.0,9,true
UnrestrictedReserveBalances,firm long,s_4_Q,1578960.0,17,encumbrance_2,826200.0,10,true


## MORPHIR
Finally, we were able to source or data through multiple JOINs operations and necessary transformations without having to write any complex SQL code. The same can be safely passed onto **Morphir** for rule based decisioning and aggregations as set by regulators. Each of those rules have been validated and unit tested through the **Morphir** framework.

In [0]:
%scala
import regulation.us.lcr.inflows.assets.{SparkJobs => Morphir}

import regulation.us.lcr.inflows.assets.{SparkJobs=>Morphir}

Let's execute our first sets of LCR rules for inflows data resulting in a new dataframe

In [0]:
%scala
val lcr = inflows.transform(Morphir.sumToRule)
display(lcr)

Label,value
20(c)(1),7.16869872E9
20(b)(1),3.51023616E9
20(a)(1),1.435490064E10


And safely append or overwrite its results onto a table. That table can be shared across different systems or organizations through delta sharing (see later) or mapped back onto legend to be accessed by end users.

In [0]:
%scala
lcr.write.format("delta").mode("overwrite").saveAsTable("lcr.report")

## TIME TRAVEL
Persisting our reports to Delta format, we benefit from its audit capability allowing users to travel back in time through all its previous versions.

In [0]:
%sql
DESCRIBE HISTORY lcr.report

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2022-11-30T09:36:08.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,3,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12
3,2022-11-29T20:00:04.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,2,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12
2,2022-11-29T19:57:29.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,1,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12
1,2022-11-29T19:57:21.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12
0,2022-11-29T19:56:53.000+0000,3658755248564160,antoine.amend@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1129-000847-nuk6je0t,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/10.4.x-photon-scala2.12


We access our reports as it was generated at a given point in time (regardless of susequent updates) or at a given version. This ensures audit and compliance requirements whilst guaranteeing strict reproducibility of our output given both our **Legend** and **Morphir** models.

In [0]:
%sql
SELECT * FROM lcr.report
VERSION AS OF 2

Label,value
20(c)(1),1.83790944E9
20(b)(1),8.6119416E8
20(a)(1),3.68918496E9


In [0]:
%sql
SELECT * FROM lcr.report
TIMESTAMP AS OF '2022-11-29T19:57:29.000+0000'

Label,value
20(c)(1),1.83790944E9
20(b)(1),8.6119416E8
20(a)(1),3.68918496E9
